<a href="https://colab.research.google.com/github/Jackfin-dev/CodeClause/blob/main/T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

# df = pd.read_csv('/content/drive/MyDrive/training.csv')
# df.head()
df_train = pd.read_csv('/content/training.csv', sep =',')

print(df_train.head())

                                               Input Sentiment
0          I can't understand the method of teaching  Negative
1  The instructor was interested in the students ...  Positive
2  The instructor don't use any examples for expl...  Negative
3          Teaching is good but always late to class   Neutral
4                               Explaination is poor  Negative


In [4]:
df_train['Sentiment'].value_counts()

Positive    576
Negative    493
Neutral     233
Name: Sentiment, dtype: int64

In [5]:
df_train['Sentiment'].isnull().values.any() #checking for anu null values

False

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
encoded_dict = {'Neutral':0,'Positive':1, 'Negative':2 }
df_train['Sentiment'] = df_train.Sentiment.map(encoded_dict)
     

In [8]:
df_train

,Input,Sentiment
0,I can't understand the method of teaching,2
1,The instructor was interested in the students ...,1
2,The instructor don't use any examples for expl...,2
3,Teaching is good but always late to class,0
4,Explaination is poor,2
...,...,...
1297,All is there except indoor games,0
1298,All is well except fitness equipments,0
1299,Everything is fine except ground condition,0
1300,Ground is in best condition but no gaming equi...,0


In [9]:
y_train = to_categorical(df_train.Sentiment)

In [10]:
y_train

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [11]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
x_train = tokenizer(
    text=df_train.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [14]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [15]:
x_train['input_ids']

<tf.Tensor: shape=(1302, 24), dtype=int32, numpy=
array([[  101,   146,  1169, ...,     0,     0,     0],
       [  101,  1109, 10332, ...,     0,     0,     0],
       [  101,  1109, 10332, ...,     0,     0,     0],
       ...,
       [  101,  5268,  1110, ...,     0,     0,     0],
       [  101,  8149,  1110, ...,     0,     0,     0],
       [  101,  1302,  4778, ...,     0,     0,     0]], dtype=int32)>

In [16]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [17]:
max_len = 24
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
     

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 24)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 24)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 24,                                            

In [18]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [19]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,y = y_train,epochs=15,batch_size=36)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


37/37 [==============================] - 390s 9s/step - loss: 0.7154 - balanced_accuracy: 0.6966
Epoch 2/15
37/37 [==============================] - 347s 9s/step - loss: 0.3956 - balanced_accuracy: 0.8687
Epoch 3/15
37/37 [==============================] - 347s 9s/step - loss: 0.2640 - balanced_accuracy: 0.9224
Epoch 4/15
37/37 [==============================] - 347s 9s/step - loss: 0.2201 - balanced_accuracy: 0.9217
Epoch 5/15
37/37 [==============================] - 347s 9s/step - loss: 0.1694 - balanced_accuracy: 0.9416
Epoch 6/15
37/37 [==============================] - 345s 9s/step - loss: 0.1444 - balanced_accuracy: 0.9485
Epoch 7/15
37/37 [==============================] - 337s 9s/step - loss: 0.1067 - balanced_accuracy: 0.9608
Epoch 8/15
37/37 [==============================] - 338s 9s/step - loss: 0.0969 - balanced_accuracy: 0.9654
Epoch 9/15
37/37 [==============================] - 338s 9s/step - loss: 0.0832 - balanced_accuracy: 0.9670
Epoch 10/15
37/37 [====================

In [20]:
texts = input(str('input the text:'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=24,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the text:nm
1/1 [==============================] - 3s 3s/step
Neutral 95.4978
Positive 2.375339
Negative 2.1268566
